# Solving equations
Click [here](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fberkeley-physics%2FPython-Tutorials&urlpath=tree%2FPython-Tutorials%2F3+-+Specific+topics%2FSolving+equations.ipynb&branch=master) to open this notebook in the DataHub.

## Learning objectives
By the end of this tutorial, you will be able to:
- Implement a basic equation solver
- Numerically solve an equation
- Numerically optimize a function

## Relevant documentation
- [Scipy `optimization` module](https://docs.scipy.org/doc/scipy/reference/optimize.html)

## Solving equations numerically
Often we'd like to solve equations, but the solutions may not be elementary. It is sometimes useful to solve such equations numerically, to obtain numerical approximations to solutions. However, this is incovenient for parametrised equations since we can find numerical solutions to only a single choice of parameter values at a time.

The general problem is, given a function $f(x)$, to find a value for $x$ such that $$f(x)=0.$$ Let's assume that a unique solution exists. 

A simple strategy, called _Newton's method,_ is to keep updating our guess for the solution by linearly extrapolating the function using its value and derivative at our current guess, i.e. $$x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)},$$ until we reach a desired accuracy. The function below implements this.

In [ ]:
import numpy as np

def newton(f, df, x0=0, tol=1e-6):
    """
    f: function of one variable
    df: derivative of f
    x0: initial guess
    tol: termination condition, indicating scale of allowed error
    returns: approximate x where f(x)=0
    """
    x = x0 #our guess
    x_update = 2*tol #our update to guess at each step (dummy value to start with)
    while abs(x_update) > tol: #terminates if our update is smaller than given tolerance
        x_update = f(x)/df(x) #calculate update
        x -= x_update #update guess
    return x

Test it out on the following function. Does it give the expected answer? Try changing the function. What happens if there are no solutions, or multiple solutions? (Try with different initial guesses `x0`. You will have to interrupt the kernel when it gets stuck in a loop, or add a maximum number of steps to give up after.)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def f(x):
    return np.exp(x) - x**2

def df(x):
    return np.exp(x) - 2*x

soln = newton(f, df)
print(soln)

x_mesh = np.linspace(-1,1,1000) + soln
plt.plot(x_mesh, f(x_mesh))
plt.axvline(soln, c="r")
plt.axhline(0,c="k")
plt.xlabel("$x$")
plt.ylabel("$f(x)$")

There are several more advanced equation solvers in the `scipy.optimize` module. Here we solve the same thing, using one of the algorithms implemented in the `root_scalar` function ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar)). There are many algorithms to choose from, which can require a bracket that is guaranteed to contain the solution, and/or the first and second derviatives of the function (which can be numerically approximated, but analytic gradients are always preferable if feasible). The following table, from the scipy documentation linked above, summarises the different algorithms and required information:
![scalar root-finding algorithms in scipy](scipy_root_scalar_algos.png)
We'll use Newton's method again, but feel fry to try out other algorithms. The solution is returned as a `RootResults` object ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.RootResults.html#scipy.optimize.RootResults)), which contains all the information about the results, including whether it has converged, and what the solution is (see the tutorial on objects to find out what an object is).

In [ ]:
from scipy import optimize

soln = optimize.root_scalar(f, x0=0, fprime=df, method="newton")
if soln.converged:
    print(soln.root)

Solving multi-dimensional equations is trickier because there is "more" space to search in higher dimensions, and because numerical derivatives are more expensive in more dimensions. You can use the `root` function ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html)), which has a similar syntax, and returns a `OptimizeResult` object ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult)). Here it is applied to the same function (estimating the Jacobian numerically).

In [ ]:
soln = optimize.root(f, x0=0)
if soln.success:
    print(soln.x)

Practice applying the knowledge you have gained by writing a function that inverts the function $f(x)=\log x + e^x.$

In [ ]:
def f(x):
    return np.log(x) + np.exp(x)

def finv(y):
    """Returns x such that f(x) = y"""
    #fill me in

## Optimisation
Sometimes, especially if the problem is overconstrained, $f(x)$ doesn't cross zero, i.e. doesn't have a solution. In this case, we can still ask, what is the closest we can come to the solution, i.e. at what $x$ is $f(x)$ closest to zero? This leads us to the more general problem of optimisation (which also explains why the root-finding algorithms are implemented in scipy's `optimization` module).

The general problem is to find the $x$ that minimises $f(x)$ (or maximises, which is equivalent by negation). Local optimisation is relatively easy: we just need to find a point at which $f'(x)=0$ (which is equaivalent to root-finding). The usual strategy is to follow the gradient downward, until we reach a minimum, but several other strategies exist. Several of them are implemented in the `minimize_scalar` ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html#scipy.optimize.minimize_scalar)) and `minimize` ([docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize)) functions. The docs explain the various algorithms.

In [ ]:
def g(x):
    return f(x)**2

soln = optimize.minimize(g, x0=0)
if soln.success:
    print(soln.x)

Global optimisation is more difficult: consider the following function. It has a small well with a deep global minimum, which can be totally unnoticed if we never sample the function in that region. Such problems become much worse in higher dimensions.

In [ ]:
def f(x):
    return x**2 - np.exp(-1e4*(x-0.5)**2)

eg_x = np.linspace(-1,1,1000)
plt.plot(eg_x, f(eg_x))
plt.xlabel("$x$")
plt.ylabel("$f(x)$")

There are some advanced algorithms in the module that allow you to do global optimisation, but they are not perfect. The following cell applies one of these algorithms to the above function. Is it successful in finding the global minimum? Try with the other global optimisation algorithms in the module (read the [docs](https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization)). Which are successful? Which are efficient? (Some of these algorithms are non-deterministic, so run them a few times each.)

In [ ]:
optimize.basinhopping(f, x0=0)